In [ ]:
import pandas as pd
import numpy as np
from combat.pycombat import pycombat
from scipy import stats
from lifelines.datasets import load_waltons

In [ ]:
data = pd.read_excel ('C:\\Users\\Mrl3\\Documents\\UZOP2022\\metabolicki_geni.xls', sheet_name='All metabolic genes')
metabolic_genes = data[['Gene Symbol']]
metabolic_genes = metabolic_genes.rename(columns = {'Gene Symbol' : 'gene_name'})
print(metabolic_genes)

     gene_name
0        ABCA1
1       ABCA10
2       ABCA12
3       ABCA13
4        ABCA2
...        ...
2747      PDXP
2748  PHOSPHO2
2749      PNPO
2750      DCXR
2751      XYLB

[2752 rows x 1 columns]


In [ ]:
data = pd.read_csv('C:\\Users\\Mrl3\\Documents\\UZOP2022\\clinical.tsv', sep='\t')
TCGA_clinical = data[['case_submitter_id', 'days_to_death']]
TCGA_clinical.drop_duplicates()
TCGA_clinical = TCGA_clinical.rename(columns = {'case_submitter_id' : 'case_id'})
print(TCGA_clinical)

          case_id days_to_death
0    TCGA-WQ-A9G7           '--
1    TCGA-WQ-A9G7           '--
2    TCGA-RC-A6M3           '--
3    TCGA-RC-A6M3           '--
4    TCGA-DD-AAE6           '--
..            ...           ...
737  TCGA-CC-5258           129
738  TCGA-BC-A10S          1423
739  TCGA-BC-A10S          1423
740  TCGA-CC-5264           102
741  TCGA-CC-5264           102

[742 rows x 2 columns]


In [ ]:
TCGA_sample_sheet = pd.read_csv('C:\\Users\\Mrl3\\Documents\\UZOP2022\\gdc_sample_sheet.2022-11-11.tsv', sep='\t')
TCGA_sample_sheet = TCGA_sample_sheet[['File ID', 'File Name', 'Case ID']]
TCGA_sample_sheet = TCGA_sample_sheet.rename(columns = {'Case ID' : 'case_id'})
print(TCGA_sample_sheet)

                                  File ID  \
0    d7e635d5-7330-491c-986c-f2737ec5d7bd   
1    772bdab0-f31c-441e-b79e-240fd622079b   
2    0829bb65-1cde-461c-b511-3649d97c62e7   
3    fcf07cf2-437a-4178-a405-c34d800cf75f   
4    ddeb287b-8d2f-40f2-89dc-def570237300   
..                                    ...   
419  e5c65ba0-d937-4ed9-aa81-99bc48717a2d   
420  28c03921-6a64-4ce6-9627-8734812e4809   
421  9973d6b0-942f-4726-8ac3-ed0ba00f6bd3   
422  4341af2d-dee1-4b4a-ad27-00858396c886   
423  50510cac-4961-4b8c-b24b-06bfbfead355   

                                             File Name       case_id  
0    9953447a-4d33-4a53-8395-4ac0fa67ed42.rna_seq.a...  TCGA-DD-AACJ  
1    3a390e78-ce52-4743-898c-8c529b342cad.rna_seq.a...  TCGA-DD-A4NE  
2    2a3e3e31-ec7e-42e3-b219-2bc84f3888ac.rna_seq.a...  TCGA-ED-A8O6  
3    0ac55bd1-ee36-4795-b9d9-4a07495e5a46.rna_seq.a...  TCGA-MI-A75I  
4    23a791d2-b0fd-4c72-b3e8-ab3ecfb3c4b8.rna_seq.a...  TCGA-FV-A3I0  
..                               

In [ ]:
data_raw = []
data= []

i=0
for col,row in TCGA_sample_sheet.iterrows():
    t = pd.read_csv('C:\\Users\\Mrl3\\Documents\\UZOP2022\\GNC\\' + row['File ID'] + "\\" + row["File Name"], delimiter="\t", header=1, usecols = ['gene_id', 'gene_name', 'tpm_unstranded'])

    tmp = pd.merge(metabolic_genes, t)
    data_raw.append(tmp)
    
    tmp=tmp.drop('gene_id', axis=1).rename(columns={'tpm_unstranded':row['case_id']})
    
    if(i == 0):
        i+ = 1
    else:
        tmp = tmp.drop('gene_name', axis=1)
    
    data.append(tmp)
    


In [1]:
TCGA = pd.concat(data, axis=1)
TCGA = TCGA.set_index('gene_name')

NameError: ignored

In [ ]:
ICGC_exp_seq = pd.read_csv('C:\\Users\\Mrl3\\Documents\\UZOP2022\\exp_seq.tsv', sep='\t')
ICGC_exp_seq = ICGC_exp_seq[['icgc_donor_id', 'gene_id', 'raw_read_count']]
ICGC_exp_seq = ICGC_exp_seq.rename(columns = {'gene_id' : 'gene_name'})
ICGC_exp_seq = pd.merge(left=ICGC_exp_seq, right=metabolic_genes)
print(ICGC_exp_seq)

        icgc_donor_id gene_name  raw_read_count
0             DO50855     SGPP1          193710
1             DO50855     SGPP1          192339
2             DO50857     SGPP1          195352
3             DO50857     SGPP1           80305
4             DO50859     SGPP1           53725
...               ...       ...             ...
1090222       DO48743   SLC18A3            2020
1090223       DO48723   SLC18A3             202
1090224       DO23547   SLC18A3             202
1090225       DO23513   SLC18A3             168
1090226       DO23508   SLC18A3            2386

[1090227 rows x 3 columns]


In [ ]:
ICGC_exp_seq['length'] = ICGC_exp_seq['gene_name'].str.len()
ICGC_exp_seq['RPK'] = ICGC_exp_seq['raw_read_count']/ICGC_exp_seq['length']
RPK_total = ICGC_exp_seq['RPK'].sum()
RPK_total = RPK_total / h1000000
ICGC_exp_seq['TPM'] = (ICGC_exp_seq['raw_read_count']/RPK_total)
ICGC_exp_seq = ICGC_exp_seq[['icgc_donor_id','gene_name','TPM']]
print(ICGC_exp_seq)

        icgc_donor_id gene_name       TPM
0             DO50855     SGPP1  1.669828
1             DO50855     SGPP1  1.658010
2             DO50857     SGPP1  1.683983
3             DO50857     SGPP1  0.692249
4             DO50859     SGPP1  0.463123
...               ...       ...       ...
1090222       DO48743   SLC18A3  0.017413
1090223       DO48723   SLC18A3  0.001741
1090224       DO23547   SLC18A3  0.001741
1090225       DO23513   SLC18A3  0.001448
1090226       DO23508   SLC18A3  0.020568

[1090227 rows x 3 columns]


In [ ]:
ICGC_donors = pd.read_csv('C:\\Users\\Mrl3\\Documents\\UZOP2022\\donor.tsv', sep='\t')

In [ ]:
ICGC = ICGC_exp_seq.pivot_table(columns='icgc_donor_id', values='TPM', index='gene_name')
ICGC

icgc_donor_id,DO227643,DO227801,DO23508,DO23509,DO23510,DO23511,DO23512,DO23513,DO23514,DO23515,...,DO50829,DO50832,DO50839,DO50840,DO50844,DO50845,DO50850,DO50855,DO50857,DO50859
gene_name,,,,,,,,,,,,,,,,,,,,,
A1CF,8.077129,14.490774,7.940282,11.172401,11.031102,4.271600,10.491781,7.868743,17.691812,11.451766,...,1.458270,22.803007,3.627736,13.035844,19.578435,2.156381,19.137566,7.097851,13.690647,21.374001
A4GALT,0.033003,0.264956,0.104012,0.060385,0.097887,0.047614,0.033016,0.076811,0.125041,0.165263,...,0.217170,0.243250,0.072953,0.144980,0.106219,0.024490,0.308601,0.117804,0.129299,0.044854
A4GNT,0.001741,0.002858,0.004129,0.004353,0.008306,0.008405,0.021464,0.003625,0.011271,0.163961,...,0.007620,0.058419,NaN,0.005021,NaN,NaN,NaN,0.062691,0.010448,0.006905
AACS,0.130847,0.310338,0.815234,0.392092,0.356628,0.207554,0.278499,1.071695,0.394803,0.907350,...,1.788046,0.613520,0.105977,0.304799,1.080367,0.231204,0.359645,0.372709,0.293205,0.356137
AADAC,14.018190,24.986365,12.640567,12.264418,16.295052,5.139943,15.074972,9.655785,28.272511,20.127541,...,1.132571,18.430059,4.667554,16.644431,0.367671,2.492950,19.698714,11.206175,15.220499,14.584226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYLB,0.720498,2.303253,0.758431,1.514177,1.982058,0.993634,1.949525,1.040214,1.208990,1.523478,...,0.319828,1.892976,0.758004,1.071850,1.142019,0.165776,2.457395,0.872266,2.067411,1.918511
XYLT1,0.026227,0.102051,0.023447,0.058665,0.070091,0.037433,0.070531,0.078862,0.084461,0.134678,...,0.209722,0.183719,0.010448,0.063583,0.135855,0.004733,0.139915,0.057333,0.198585,0.024010
XYLT2,0.446322,1.108197,1.123335,0.718295,0.621438,0.872628,0.838634,1.898885,0.980855,1.377391,...,1.665983,2.441883,0.829044,0.979734,2.036094,0.352663,0.868658,0.838875,0.697236,0.890880


In [ ]:
SET = ICGC.merge(TCGA)
print(SET)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [2]:
COMBAT = pycombat(ICGC, batch)

NameError: ignored

[4.49580385 0.04932075        nan ... 0.26543035 0.23904377 0.41992027]
